In [1]:
import gta_prices
import numpy as np
import pandas as pd

In [2]:
def state(entry_signal, exit_signal, period):
    '''Calculate trade state signals.'''
    df = pd.concat([entry_signal, exit_signal], axis=1)
    df.columns = ['entry', 'exit']
    df['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'entry'] == 1 \
                and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'exit'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
    return df.state

In [3]:
# Trade parameters.
exchange = 'LSE'
tidm = 'HSV'
periods = {'p1': 48, 'p2': 24, 'p3': 12, 'p4': 6}
position_size = 7500
risk_pct = 0.2
commission = 11.95
sduty = 0.5

In [4]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm)

In [5]:
# Donchian lines, entries & exits for each look back period.
entry = None
for i, period in enumerate(periods.values()):
    i += 1
    globals()[f'p{i}'] = period
    df = pd.DataFrame(prices.copy())
    df['upr'] = prices.high.rolling(period).max().shift(periods=1)
    df['lwr'] = prices.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    df['sys'] = i
    df['period'] = period
    if i == 1:
        df['buy'] = np.where(df.close > df.upr, 1, 0)
    else:
        df['buy'] = entry
    df['sell'] = np.where(df.close < df.mid, 1, 0)
    df['state'] = state(df.buy, df.sell, period)
    df['entry'] = np.where(np.logical_and(df.state == 1, df.state.shift(periods=1) == 0), 1, 0)
    if i == 1:
        entry = df.entry
    df['exit'] = np.where(np.logical_and(df.state == 0, df.state.shift(periods=1) == 1), 1, 0)
    globals()[f'dc{i}'] = df

In [6]:
# Trade list indexed by date.
td = pd.concat([dc1[dc1.entry == 1], dc1[dc1.exit == 1], dc2[dc2.exit == 1], dc3[dc3.exit == 1],
                dc4[dc4.exit == 1]], axis=0)
td = td.sort_index()

In [7]:
# Initial volatility adjusted position size on entry.
td['volatility'] = np.where(td.entry == 1, abs((td.mid - td.close) / td.close), 0)
td['risk_amt'] = np.where(td.entry == 1, ((position_size * risk_pct) / td.volatility), 0)
td['shares'] = np.where(td.entry == 1, (td.risk_amt / td.close).astype('int'), 0)

In [8]:
# Modify number of shares to be purchased to be divisible by 4.
s = pd.Series(td.shares.copy())
for i, share in enumerate(s):
    while s.iloc[i] % 4 != 0:
        s.iloc[i] += 1
td.shares = s

In [9]:
# Adjust risk amount on entry for revised share count.
td.risk_amt = np.where(td.entry == 1, (td.close * td.shares), 0)

In [10]:
# Position size (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        shares = row['shares']
    elif row['exit'] == 1:
        td.at[index, 'shares'] = int(shares / 4)

In [11]:
td

,open,high,low,close,upr,lwr,mid,sys,period,buy,sell,state,entry,exit,volatility,risk_amt,shares
date,,,,,,,,,,,,,,,,,
2001-12-07,1.1954,1.2277,1.1954,1.2277,1.2062,0.87769,1.041945,1,48,1,0,1,1,0,0.151303,9914.9052,8076
2002-02-01,1.3462,1.3462,1.3031,1.3085,1.3591,1.27080,1.314950,4,6,0,1,0,0,1,0.000000,0.0000,2019
2002-02-22,1.3085,1.3085,1.2568,1.2568,1.3591,1.19000,1.274550,3,12,0,1,0,0,1,0.000000,0.0000,2019
2002-04-05,1.2385,1.2492,1.2094,1.2148,1.3591,1.10380,1.231450,2,24,0,1,0,0,1,0.000000,0.0000,2019
2002-09-06,1.1792,1.1846,1.1415,1.1469,1.3591,0.99077,1.174935,1,48,0,1,0,0,1,0.000000,0.0000,2019
2003-09-05,1.1254,1.1738,1.1200,1.1706,1.1415,0.78615,0.963825,1,48,1,0,1,1,0,0.176640,8493.8736,7256
2003-10-03,1.1577,1.1631,1.1523,1.1577,1.1954,1.12000,1.157700,4,6,0,1,0,0,1,0.000000,0.0000,1814
2003-10-10,1.1545,1.1545,1.0985,1.1092,1.1954,1.05320,1.124300,3,12,0,1,0,0,1,0.000000,0.0000,1814
2004-02-06,1.1825,1.1825,1.1458,1.1458,1.2385,1.09310,1.165800,2,24,0,1,0,0,1,0.000000,0.0000,1814


In [12]:
# Adjust calcs to handle any number of systems, ie. not tailored to 4.